In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
import os

In [2]:
spark = SparkSession.builder\
    .appName("Practice")\
    .master("local[*]")\
    .getOrCreate()

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
26/01/30 16:49:14 WARN Utils: Your hostname, Somnath, resolves to a loopback address: 127.0.1.1; using 10.255.255.254 instead (on interface lo)
26/01/30 16:49:14 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
26/01/30 16:49:15 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


READ MODE

In [3]:
file_path = "/home/somnath/my_vscode_project/input_data/MOCK_DATA.csv"
path = "/home/somnath/my_vscode_project/output_data/gender_bucketed_table/"
#output_path = "/home/somnath/my_vscode_project/output_data/"

In [4]:
df = spark.read \
    .format("csv") \
    .option("header", "true") \
    .option("mode", "PERMISSIVE") \
    .option("inferSchema", "true") \
    .load(file_path)

In [5]:
df.printSchema()

root
 |-- id: integer (nullable = true)
 |-- name: string (nullable = true)
 |-- age: double (nullable = true)
 |-- salary: double (nullable = true)
 |-- address: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- country: string (nullable = true)



WRITE MODE WITH BUCKET

In [6]:
df.repartition(4).write.format("CSV") \
    .option("header", "true") \
    .mode("overwrite") \
    .option("path", "/home/somnath/my_vscode_project/output_data/gender_bucketed_table") \
    .bucketBy(4, "gender") \
    .saveAsTable("gender_bucketed_table")    

In [7]:
# Show the first few rows
spark.table("gender_bucketed_table").show()

+---+---------+----+-------+------------+------+-------+
| id|     name| age| salary|     address|gender|country|
+---+---------+----+-------+------------+------+-------+
| 62|  Tymothy|41.0|43819.0|    Suite 61|     M|     UK|
| 27|     Sean|27.0|56958.0|   Room 1757|     M|    USA|
| 68|    Dolly|46.0|57604.0|  17th Floor|     F|Germany|
| 88|     Jack|48.0|47609.0|PO Box 19255|     M| Canada|
| 11|   Newton|53.0|51172.0|    Room 362|     M|     UK|
| 42| Eldredge|36.0| 7456.0|    Suite 66|     M|   NULL|
|  1| Jessamyn|41.0| 9700.0|    Apt 1265|     F|    USA|
| 21|Stillmann|30.0|47529.0|PO Box 41860|     M|     UK|
| 91|      Ira|38.0|69168.0|     Apt 632|     M|Germany|
| 43|Esmeralda|NULL|62500.0|     Apt 580|     F|     UK|
| 50|    Hanni|25.0|47748.0|   Room 1123|     F|  India|
| 54| Basilius|44.0|48885.0|PO Box 59288|     M|    USA|
| 72|    Halli|63.0|14345.0|    Room 272|     F|     UK|
| 95|    Daven|48.0|40298.0|     Suite 5|     M|   NULL|
| 53|    Teddy|50.0|48002.0|PO 

In [8]:
spark.sql("DESCRIBE EXTENDED gender_bucketed_table").show(50, truncate=False)

+----------------------------+----------------------------------------------------------------------+-------+
|col_name                    |data_type                                                             |comment|
+----------------------------+----------------------------------------------------------------------+-------+
|id                          |int                                                                   |NULL   |
|name                        |string                                                                |NULL   |
|age                         |double                                                                |NULL   |
|salary                      |double                                                                |NULL   |
|address                     |string                                                                |NULL   |
|gender                      |string                                                                |NULL   |
|country  

In [9]:
os.listdir(path)

['.part-00002-e37dee10-a479-49cf-ae0b-00de628de3e2_00000.c000.csv.crc',
 '.part-00000-e37dee10-a479-49cf-ae0b-00de628de3e2_00000.c000.csv.crc',
 'part-00000-e37dee10-a479-49cf-ae0b-00de628de3e2_00000.c000.csv',
 '_SUCCESS',
 '.part-00001-e37dee10-a479-49cf-ae0b-00de628de3e2_00002.c000.csv.crc',
 'part-00003-e37dee10-a479-49cf-ae0b-00de628de3e2_00000.c000.csv',
 'part-00001-e37dee10-a479-49cf-ae0b-00de628de3e2_00000.c000.csv',
 '.part-00001-e37dee10-a479-49cf-ae0b-00de628de3e2_00000.c000.csv.crc',
 '.part-00003-e37dee10-a479-49cf-ae0b-00de628de3e2_00000.c000.csv.crc',
 '._SUCCESS.crc',
 'part-00001-e37dee10-a479-49cf-ae0b-00de628de3e2_00002.c000.csv',
 'part-00002-e37dee10-a479-49cf-ae0b-00de628de3e2_00000.c000.csv']

In [10]:
spark.stop()